## Dendrite Segmentation

### Image Binarization

Set image sampling density in micrometer/pixel.

In [11]:
sampling_x = 0.025
sampling_y = 0.025
sampling_z = 0.1

Load dendrite image file.<br>
Pass .tif file path into *load_tif* function.

In [16]:
import numpy as np
from spine_segmentation import load_tif
from notebook_widgets import show_3d_image
from scipy.ndimage import gaussian_filter
import os

name = '3_full_res (5)'
image_path = f'data/{name}/{name}.tif'
image_name = os.path.basename(image_path)

# load tif
image: np.ndarray = load_tif(image_path)
show_3d_image(image)

interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=210), IntSlider(value=0…

<function notebook_widgets.show_3d_image.<locals>.display_slice(x, y, z)>

Perform image binarization.<br>
Parameters:<br>
* $BaseThreshold$ — base threshold value
* $Weight$ — how much neighbouring values affect threshold
* $BlockSize$ — size of the neighbourhood area to calculate median value in

Local binarization is calculated as follows:<br>

$LocalThreshold_{xyz} = BaseThreshold + Weight \cdot (BaseThreshold - LocalMedianValue_{xyz}(BlockSize))$<br>
$BinarizedValue_{xyz} = \begin{cases} 1\text{, }Value_{xyz} > LocalThreshold_{xyz} \\ 0\text{, else} \end{cases}$

In [19]:
from notebook_widgets import interactive_binarization

binarization_widget = interactive_binarization(gaussian_filter(image, 1))
display(binarization_widget)

interactive(children=(IntSlider(value=127, continuous_update=False, description='base_threshold', max=255), In…

In [20]:
from notebook_widgets import select_connected_component_widget

select_component_widget = select_connected_component_widget(binarization_widget.result)
display(select_component_widget)

interactive(children=(IntSlider(value=0, continuous_update=False, description='label_index', max=10), Output()…

#### Necks reconnection

Run if need to reconnect some components

In [7]:
%load_ext autoreload
%autoreload 2

In [21]:
from necks_reconnection import get_reconnection_widget
reconnection_widget = get_reconnection_widget(np.moveaxis(image, -1, 0), np.moveaxis(binarization_widget.result, -1, 0))
reconnection_widget.show()

Start processing
Neck 1
points: (20, 50, 213) and (20, 77,228)
rect: (46, 208) (84, 232)
Start finding path
Graph built
Finidg shortest
Path found
Start floodfill
Floodfill finished


### Construct 3D surface to perform segmention on.

First, from binarized image, calculate points that belong to the surface, as well as surface normals in those points.
Use Poisson surface reconstruction algorithm to generate the surface mesh.<br>
Algorithm takes set of points with normals and produces a smooth closed surface mesh $S$.<br>
Generated mesh is saved to <i>"output/surface_mesh.off"</i> file.

In [22]:
select_component_widget = select_connected_component_widget(np.moveaxis(reconnection_widget.layers['Saved result'].data, 0, -1))
display(select_component_widget)

interactive(children=(IntSlider(value=0, continuous_update=False, description='label_index', max=4), Output())…

In [29]:
from spine_segmentation import get_surface_points
from CGAL.CGAL_Point_set_3 import Point_set_3
from CGAL.CGAL_Polyhedron_3 import Polyhedron_3
from CGAL.CGAL_Poisson_surface_reconstruction import poisson_surface_reconstruction
from notebook_widgets import show_3d_mesh, create_dir
import os
from spine_segmentation import save_tif, apply_scale 


# extract binarization result
binary_image = select_component_widget.result

# export surface mesh to .off file
save_path = f"output/{image_name.rsplit('.', 1)[0]}"

create_dir("output")
create_dir(save_path)

save_tif(f"{save_path}/{image_name}", image)
save_tif(f"{save_path}/binarized_image.tif", binary_image)

z_display_factor = 0.5

# calculate surface points
# (only apply sampling scale after surface reconstruction, because small coordinates give worse reconstruction results smh)
# surface_points: Point_set_3 = get_surface_points(binary_image, (sampling_x, sampling_y, sampling_z))
surface_points: Point_set_3 = get_surface_points(binary_image)

# construct surface mesh
surface_poly = Polyhedron_3()
poisson_surface_reconstruction(surface_points, surface_poly)

surface_poly = apply_scale(surface_poly, (sampling_x, sampling_y, sampling_z))
surface_poly.write_to_file(f"{save_path}/surface_mesh.off")

# render surface mesh
show_3d_mesh(apply_scale(surface_poly, (1, 1, 1 * z_display_factor)))

C:\Users\Ilya\.conda\envs\sa\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(10.072584…

### Mesh segmentation.

Parameters:
* Sensitivity — how much distance from skeleton affects segmentation. Higher values will result in less false positive spines, but worse segmentation at spine base and detection of smaller spines. 


Algorith is as follows:
1. Calculate mesh skeleton. Mean Curvature Skeleton algorithm is used. Algorithm generates skeleton graph $G$ and correspondence from each point on the surface to some point in the skeleton $f:S\rightarrow G$.
2. Find dendrite skeleton subgraph $G_{dendrite}$ — longest path in the graph, with the least sum angle between consecutive edges.
3. Mark surface points that don't correspond to dendrite skeleton subgraph as spines. $S_{spines} = \{ p \mid p \in S \land f(p) \notin G_{dendrite} \}$
4. Calculate distance from surface to skeleton statistic. $Dist = \{ \| p-f(p) \| \mid p \in S \}$
5. Mark surface points that are futher away from skeleton than others as spines. $S_{spines} = S_{spines} \cup \{ p \mid p \in S \land \| p - f(p) \| > quantile(Dist, Sensitivity) \}$

In [30]:
from CGAL.CGAL_Surface_mesh_skeletonization import surface_mesh_skeletonization
from CGAL.CGAL_Polygon_mesh_processing import Polylines
from spine_segmentation import build_graph, build_correspondence, build_reverse_correpondnce
from notebook_widgets import interactive_segmentation

# reload mesh because saving as .off compresses point coordinates
# and we want exactly same coordinates for spines and denrite
surface_poly = Polyhedron_3(f"{save_path}/surface_mesh.off")
surface_poly = apply_scale(surface_poly, (1, 1, 1 * z_display_factor))

# get surface skeleton
skeleton_polylines = Polylines()
correspondence_polylines = Polylines()
surface_mesh_skeletonization(surface_poly, skeleton_polylines, correspondence_polylines)

# convert to more performant data format 
skeleton_graph = build_graph(skeleton_polylines)
corr = build_correspondence(correspondence_polylines)
reverse_corr = build_reverse_correpondnce(correspondence_polylines)

# perform segmentation 
segmentation_widget = interactive_segmentation(surface_poly, corr, reverse_corr, skeleton_graph)
display(segmentation_widget)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(10.072599…

interactive(children=(FloatLogSlider(value=0.001, continuous_update=False, description='sensitivity', max=0.0,…

Generate surface mesh for each individual spine.<br>
Calculate metrics for each spine. Calculated metric names are defined in the *metric_names* list.<br>
Manually remove false positive spine selections.<br>
Use **index** to navigate spines, **checkbox** to keep or remove spine.

In [31]:
from spine_segmentation import get_spine_meshes
from spine_metrics import calculate_metrics
from notebook_widgets import select_spines_widget

# extract segmentation result
segmentation = segmentation_widget.result

# extract spine meshes
spine_meshes = get_spine_meshes(surface_poly, segmentation)

# define calculated metrics
metric_names = ["Area", "Volume"]

# manually select valid spines
selection_widget = select_spines_widget(spine_meshes, surface_poly, metric_names)
display(selection_widget)

Generate final segmentation from manually filtered spines. Spine meshes are saved to <i>"output/spine_{i}.off"</i> files.

In [32]:
from spine_segmentation import spines_to_segmentation, save_segmentation 
from notebook_widgets import show_segmented_mesh


# extract selected spines
selected_spines = []

# export selected spine meshes to .off files
for i, (spine_mesh, spine_metrics) in enumerate(selection_widget.children[1].result):
    filename = f"{save_path}/spine_{i}.off"
    scaled_spine_mesh = apply_scale(spine_mesh, (1, 1, 1 / z_display_factor))  
    scaled_spine_mesh.write_to_file(filename)
    selected_spines.append(spine_mesh)

# combine selected spines into segmentation
manually_filtered_segmentation = spines_to_segmentation(selected_spines)
save_segmentation(manually_filtered_segmentation, f"{save_path}/segmentation.json")

# render manually filtered segmentation
show_segmented_mesh(surface_poly, manually_filtered_segmentation)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(10.072599…